In [0]:
train_y=train_y.reshape(-1,1)

In [2]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
import tensorflow as tf
tf.enable_eager_execution()

Using TensorFlow backend.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:

import numpy as np
import pandas as pd

# fix random seed for reproducibility
np.random.seed(7)
# Read the dataset in a new python notebook.
dataset = pd.read_csv("/content/drive/My Drive/R6/R6/R6-Project1/bank.csv")

In [0]:
# Drop the columns which are unique for all users like IDs 
dataset.drop("Surname", axis=1, inplace=True)
dataset.drop("RowNumber", axis=1, inplace=True)
dataset.drop("CustomerId", axis=1, inplace=True)
dataset.drop("Geography", axis=1, inplace=True)
dataset.drop("Gender", axis=1, inplace=True)

In [6]:
dataset.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,42,2,0.00,1,1,1,101348.88,1
1,608,41,1,83807.86,1,0,1,112542.58,0
2,502,42,8,159660.80,3,1,0,113931.57,1
3,699,39,1,0.00,2,0,0,93826.63,0
4,850,43,2,125510.82,1,1,1,79084.10,0


In [0]:
#convert dataset to numpy array
data=dataset.to_numpy()

In [8]:
type(data)

numpy.ndarray

**feature set: ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary' ]**

**target set: ['Exited']**

In [0]:
#Distinguish the feature and target set (2.5 points)
X = data[:,0:8]
Y = data[:,8]
#Divide the data set into Train and test sets
train_x,test_x,train_y,test_y=train_test_split(X,Y,test_size=.30,random_state=2)

In [0]:
train_x =np.array(train_x).astype('float32')
test_x = np.array(test_x).astype('float32')
train_y =np.array(train_y).astype('float32')
test_y = np.array(test_y).astype('float32')

In [0]:
#Normalize the train and test data
from sklearn.preprocessing import Normalizer

transformer = Normalizer()
train_x = transformer.fit_transform(train_x)
test_x = transformer.transform(test_x)

In [12]:
#Initialize &amp; build the model (
  train_y.shape

(7000,)

In [0]:
train_y=train_y.reshape(-1,1)

In [14]:
train_y

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [1.],
       [0.]], dtype=float32)

In [0]:
#We are initializing weights and Bias of (Input-Hidden Layer)
w1 = tf.random_normal(shape=(8,5))
b1 = tf.zeros(shape=(5))

In [0]:
#We are initializing weights and Bias of (Hidden-Output)
w2 = tf.random_normal(shape=(5,1))
b2 = tf.zeros(shape=(1))   # what is zeroes - zero initialization 

In [0]:
def prediction(x, w1, b1,w2,b2):
    
    xw_matmul = tf.matmul(x, w1)
    net1 = tf.add(xw_matmul, b1)
    y=tf.sigmoid(net1)
    net2=tf.matmul(y, w2)+b2
    out=tf.sigmoid(net2)
    return net2,out # for regression use net2 and classification use out

In [0]:
def train(train_x, train_y, w1, b1,w2,b2, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w1,b1,w2,b2]) #will track the parameters watch it
        
        net2,current_prediction = prediction(train_x, w1, b1,w2,b2)
        current_loss =tf.nn.sigmoid_cross_entropy_with_logits(labels = train_y,logits =net2 ) ##internally calculates oute using net2 hence we are using net2 even though this is a classification problem
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw1,db1,dw2,db2 = t.gradient(current_loss,[w1, b1,w2,b2])
    
    #Update Weights at output layer
    w2 = w2 - learning_rate*dw2
    b2 = b2 - learning_rate*db2
    
     #Update Weights at hidden layer
    w1 = w1 - learning_rate*dw1
    b1 = b1 - learning_rate*db1
    
    return w1, b1,w2,b2

In [21]:
for i in range(500):
    
    w1, b1,w2,b2 = train(train_x, train_y, w1, b1,w2,b2)
    

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [22]:
w1

<tf.Tensor: id=34523, shape=(8, 5), dtype=float32, numpy=
array([[ 0.6446062 , -0.9904006 , -1.3077853 ,  0.66945165,  1.2159653 ],
       [ 0.62995684, -0.19765283,  0.42947444, -0.15706012,  0.09475265],
       [-2.4200842 , -0.32517236, -0.46426404, -0.26610407, -0.27927607],
       [-1.9898428 , -2.8496087 , -1.4218982 , -1.996314  , -4.7006598 ],
       [-0.01441761, -0.05910248,  0.7541601 ,  1.1404089 ,  0.93126667],
       [-0.90542257, -1.0293137 ,  2.248236  , -0.13004856, -0.9749662 ],
       [ 1.2957394 , -1.1071007 , -0.1750002 , -1.8053511 ,  1.1584802 ],
       [-4.4042773 , -2.5857964 , -4.341414  , -3.1434247 , -4.522531  ]],
      dtype=float32)>

In [23]:
b1

<tf.Tensor: id=34526, shape=(5,), dtype=float32, numpy=
array([-5.2434077, -3.2752922, -3.2199085, -6.3726645, -6.44199  ],
      dtype=float32)>

In [24]:
w2

<tf.Tensor: id=34517, shape=(5, 1), dtype=float32, numpy=
array([[ 9.304078  ],
       [-0.88398445],
       [ 1.3304532 ],
       [ 8.24065   ],
       [ 8.238414  ]], dtype=float32)>

In [25]:
b2

<tf.Tensor: id=34520, shape=(1,), dtype=float32, numpy=array([-19.067799], dtype=float32)>

In [26]:
_,y_pred=prediction(test_x,w1,b1,w2,b2)
out=[]
for val in y_pred:
  if val>np.mean(y_pred):
    out.append(1)
  else:
    out.append(0)
print(out)
      

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [0]:
from sklearn import metrics

In [28]:
cm=metrics.classification_report(test_y,net1)
print(cm)

              precision    recall  f1-score   support

         0.0       0.81      1.00      0.89      2415
         1.0       0.00      0.00      0.00       585

    accuracy                           0.81      3000
   macro avg       0.40      0.50      0.45      3000
weighted avg       0.65      0.81      0.72      3000



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
